<a href="https://colab.research.google.com/github/ItsNotRohit02/Attendance-System-using-Facial-Recognition/blob/main/6_PytorchModularCellMode.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [25]:
import torch
from torch import nn

from torch.utils.data import DataLoader
from torchvision import datasets, transforms

from timeit import default_timer as timer
from tqdm.auto import tqdm
from pathlib import Path

import requests
import os
import zipfile

In [26]:
device = "cuda" if torch.cuda.is_available() else "cpu"
device

'cuda'

In [27]:
data_path = Path("data/")
image_path = data_path / "pizza_steak_sushi"

if image_path.is_dir():
  print(f"{image_path} already exists")

else:
  print(f"{image_path} created")
  image_path.mkdir(parents=True, exist_ok=True)

  with open(data_path / "pizza_steak_sushi.zip", "wb") as f:
    request = requests.get("https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip")
    print("Downloading")
    f.write(request.content)

  with zipfile.ZipFile(data_path / "pizza_steak_sushi.zip", "r") as zip_ref:
    zip_ref.extractall(image_path)
    print("Downloaded")

  os.remove(data_path / "pizza_steak_sushi.zip")

data/pizza_steak_sushi already exists


In [28]:
train_dir = image_path / "train"
test_dir = image_path / "test"

In [29]:
simple_transform = transforms.Compose([transforms.Resize(size=(64,64)),
                                       transforms.ToTensor()])

In [30]:
train_data = datasets.ImageFolder(root = train_dir,
                                  transform = simple_transform,
                                  target_transform=None)

test_data = datasets.ImageFolder(root = test_dir,
                                  transform = simple_transform)

In [31]:
train_dataloader = DataLoader(dataset= train_data, batch_size= 32,
                              num_workers=os.cpu_count(),
                              shuffle=True)

test_dataloader = DataLoader(dataset= test_data, batch_size= 32,
                              num_workers=os.cpu_count(),
                              shuffle=False)

In [32]:
class_names = train_data.classes
class_names

['pizza', 'steak', 'sushi']

In [33]:
class_dict = train_data.class_to_idx
class_dict

{'pizza': 0, 'steak': 1, 'sushi': 2}

In [34]:
img_batch, label_batch = next(iter(train_dataloader))
img_batch.shape, label_batch.shape

(torch.Size([32, 3, 64, 64]), torch.Size([32]))

In [35]:
class TinyVGGModel(nn.Module):
  def __init__(self, input_shape:int, hidden_units:int, output_shape:int):
    super().__init__()
    self.block1 = nn.Sequential(
        nn.Conv2d(in_channels=input_shape,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
    )
    self.block2 = nn.Sequential(
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.Conv2d(in_channels=hidden_units,
                  out_channels=hidden_units,
                  kernel_size=3,
                  stride=1,
                  padding=0),
        nn.ReLU(),
        nn.MaxPool2d(kernel_size=2)
      )
    self.classifier = nn.Sequential(
        nn.Flatten(),
        nn.Linear(in_features=hidden_units*13*13, out_features=output_shape))
  def forward(self, x:torch.Tensor) -> torch.Tensor:
    #x = self.block1(x)
    #print(f"Output from block1:{x.shape}")
    #x = self.block2(x)
    #print(f"Output from block2:{x.shape}")
    #x = self.classifier(x)
    #return x
    return self.classifier(self.block2(self.block1(x)))

In [36]:
model0 = TinyVGGModel(input_shape=3, hidden_units=10, output_shape=len(class_names)).to(device)

In [37]:
model0.eval()
with torch.inference_mode():
    pred = model0(img_batch.to(device))
pred

tensor([[ 0.0255, -0.0160, -0.0271],
        [ 0.0222, -0.0138, -0.0193],
        [ 0.0237, -0.0173, -0.0215],
        [ 0.0238, -0.0166, -0.0233],
        [ 0.0235, -0.0147, -0.0246],
        [ 0.0225, -0.0148, -0.0262],
        [ 0.0243, -0.0152, -0.0232],
        [ 0.0231, -0.0131, -0.0257],
        [ 0.0220, -0.0160, -0.0219],
        [ 0.0229, -0.0146, -0.0239],
        [ 0.0235, -0.0184, -0.0251],
        [ 0.0256, -0.0146, -0.0247],
        [ 0.0222, -0.0146, -0.0246],
        [ 0.0251, -0.0160, -0.0245],
        [ 0.0255, -0.0170, -0.0268],
        [ 0.0260, -0.0132, -0.0238],
        [ 0.0239, -0.0143, -0.0235],
        [ 0.0237, -0.0141, -0.0256],
        [ 0.0233, -0.0147, -0.0256],
        [ 0.0217, -0.0169, -0.0237],
        [ 0.0277, -0.0154, -0.0278],
        [ 0.0220, -0.0150, -0.0229],
        [ 0.0233, -0.0154, -0.0252],
        [ 0.0212, -0.0167, -0.0255],
        [ 0.0253, -0.0150, -0.0254],
        [ 0.0240, -0.0163, -0.0265],
        [ 0.0225, -0.0166, -0.0224],
 

In [38]:
def train_step(model: torch.nn.Module,
               dataloader: torch.utils.data.DataLoader,
               loss_fn: torch.nn.Module,
               optimizer: torch.optim.Optimizer):
    # Put model in train mode
    model.train()

    # Setup train loss and train accuracy values
    train_loss, train_acc = 0, 0

    # Loop through data loader data batches
    for batch, (X, y) in enumerate(dataloader):
        # Send data to target device
        X, y = X.to(device), y.to(device)

        # 1. Forward pass
        y_pred = model(X)

        # 2. Calculate  and accumulate loss
        loss = loss_fn(y_pred, y)
        train_loss += loss.item()

        # 3. Optimizer zero grad
        optimizer.zero_grad()

        # 4. Loss backward
        loss.backward()

        # 5. Optimizer step
        optimizer.step()

        # Calculate and accumulate accuracy metric across all batches
        y_pred_class = torch.argmax(torch.softmax(y_pred, dim=1), dim=1)
        train_acc += (y_pred_class == y).sum().item()/len(y_pred)

    # Adjust metrics to get average loss and accuracy per batch
    train_loss = train_loss / len(dataloader)
    train_acc = train_acc / len(dataloader)
    return train_loss, train_acc

In [39]:
def test_step(model: torch.nn.Module,
              dataloader: torch.utils.data.DataLoader,
              loss_fn: torch.nn.Module):
    # Put model in eval mode
    model.eval()

    # Setup test loss and test accuracy values
    test_loss, test_acc = 0, 0

    # Turn on inference context manager
    with torch.inference_mode():
        # Loop through DataLoader batches
        for batch, (X, y) in enumerate(dataloader):
            # Send data to target device
            X, y = X.to(device), y.to(device)

            # 1. Forward pass
            test_pred_logits = model(X)

            # 2. Calculate and accumulate loss
            loss = loss_fn(test_pred_logits, y)
            test_loss += loss.item()

            # Calculate and accumulate accuracy
            test_pred_labels = test_pred_logits.argmax(dim=1)
            test_acc += ((test_pred_labels == y).sum().item()/len(test_pred_labels))

    # Adjust metrics to get average loss and accuracy per batch
    test_loss = test_loss / len(dataloader)
    test_acc = test_acc / len(dataloader)
    return test_loss, test_acc

In [40]:
def train(model: torch.nn.Module,
          train_dataloader: torch.utils.data.DataLoader,
          test_dataloader: torch.utils.data.DataLoader,
          loss_fn: torch.nn.Module,
          optimizer: torch.optim.Optimizer,
          epochs:int=2,
          device=device):

  results = {"train_loss": [],
             "train_acc": [],
             "test_loss": [],
             "test_acc": []}

  for epoch in tqdm(range(epochs)):

    train_loss, train_acc = train_step(model, train_dataloader, loss_fn, optimizer)
    test_loss, test_acc = train_step(model, train_dataloader, loss_fn, optimizer)

    print(f"Epoch:{epoch} | Train Loss: {train_loss:.4f} | Train Accuracy: {train_acc:.4f} | Test Loss: {test_loss:.4f} | Test Accuracy: {test_acc:.4f} ")

    results["train_loss"].append(train_loss)
    results["train_acc"].append(train_acc)
    results["test_loss"].append(test_loss)
    results["test_acc"].append(test_acc)
  return results

In [41]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(params=model0.parameters(), lr=0.001)

In [42]:
def save_model(model: torch.nn.Module,
               target_dir: str,
               model_name: str):

  # Create target directory
  target_dir_path = Path(target_dir)
  target_dir_path.mkdir(parents=True,
                        exist_ok=True)

  # Create model save path
  assert model_name.endswith(".pth") or model_name.endswith(".pt"), "model_name should end with '.pt' or '.pth'"
  model_save_path = target_dir_path / model_name

  # Save the model state_dict()
  print(f"[INFO] Saving model to: {model_save_path}")
  torch.save(obj=model.state_dict(),
             f=model_save_path)

In [43]:
torch.manual_seed(42)
torch.cuda.manual_seed(42)

NUM_EPOCHS = 5

start = timer()

model0_res = train(model=model0,
                   train_dataloader=train_dataloader,
                   test_dataloader=test_dataloader,
                   optimizer=optimizer,
                   loss_fn=loss_fn,
                   epochs=NUM_EPOCHS,
                   device=device)

end = timer()
print(f"[INFO] Total training time: {end-start:.3f} seconds")

save_model(model=model0,
           target_dir="models",
           model_name="05_going_modular_cell_mode_tinyvgg_model.pth")

  0%|          | 0/5 [00:00<?, ?it/s]

Epoch:0 | Train Loss: 1.1019 | Train Accuracy: 0.2500 | Test Loss: 1.0846 | Test Accuracy: 0.4258 
Epoch:1 | Train Loss: 1.0741 | Train Accuracy: 0.4258 | Test Loss: 1.0720 | Test Accuracy: 0.4258 
Epoch:2 | Train Loss: 1.1265 | Train Accuracy: 0.3047 | Test Loss: 1.0987 | Test Accuracy: 0.3047 
Epoch:3 | Train Loss: 1.0840 | Train Accuracy: 0.5234 | Test Loss: 1.0798 | Test Accuracy: 0.4219 
Epoch:4 | Train Loss: 1.0710 | Train Accuracy: 0.4062 | Test Loss: 1.0279 | Test Accuracy: 0.5430 
[INFO] Total training time: 9.641 seconds
[INFO] Saving model to: models/05_going_modular_cell_mode_tinyvgg_model.pth
